In [113]:
### Limpieza del Archivo de Medicamentos, incluye calculos de variables necesarias para 
### calcular la adherencia

In [114]:
import pandas as pd
from datetime import timedelta
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer,MissingIndicator 

medicamentos=pd.read_excel("../datos originales/Medicamentos.xlsx")

In [115]:
### Renombramiento de columnas segun el estilo acordado



In [116]:
medicamentos.columns=[x.upper() for x in medicamentos.columns ]
medicamentos.rename(columns={"ID":"ID_PACIENTE"},inplace=True)
medicamentos.rename(columns={"DESCRIPCION_PRESTACION":"NOMBRE_MEDICAMENTO"},inplace=True)

In [117]:
medicamentos.columns

Index(['ID_PACIENTE', 'FECHA_EMISION', 'REGIONAL_EPS_DESC',
       'CODIGO_PRESTACION_OP', 'NOMBRE_MEDICAMENTO',
       'CODIGO_DIAGNOSTICO_EPS_OP', 'DIAGNOSTICO_EPS_DESC',
       'NUMERO_CANTIDAD_PRESTACIONES'],
      dtype='object')

In [118]:
### Revision Columnas

## ID Paciente
print(medicamentos.ID_PACIENTE.drop_duplicates().count())
print(medicamentos.ID_PACIENTE.isna().sum())
print(medicamentos.ID_PACIENTE.isnull().sum())

1267
0
0


In [119]:
### Fecha Emision

medicamentos.FECHA_EMISION=pd.to_datetime(medicamentos.FECHA_EMISION)
print(medicamentos.FECHA_EMISION.min())
print(medicamentos.FECHA_EMISION.max())
print(medicamentos.FECHA_EMISION.isna().sum())
print(medicamentos.FECHA_EMISION.isnull().sum())

2016-12-01 00:00:00
2020-05-18 00:00:00
0
0


In [120]:
### Regional Eps Descr

medicamentos.REGIONAL_EPS_DESC.drop_duplicates()

0           Medellin
7       Barranquilla
19            Bogota
59              Cali
67         Manizales
148          Armenia
249          Pereira
5628       Cartagena
Name: REGIONAL_EPS_DESC, dtype: object

In [121]:
### Codigo de presptacion OP

medicamentos.CODIGO_PRESTACION_OP.drop_duplicates().count()
medicamentos.CODIGO_PRESTACION_OP.isna().sum()
medicamentos.CODIGO_PRESTACION_OP.isnull().sum()
## 1491

0

In [122]:
### Descricion prestacion/Nombre del Medicamento 

##'CODIGO_DIAGNOSTICO_EPS_OP', 'DIAGNOSTICO_EPS_DESC',
##       'NUMERO_CANTIDAD_PRESTACIONES']

print(medicamentos.NOMBRE_MEDICAMENTO.drop_duplicates().count())
print(medicamentos.NOMBRE_MEDICAMENTO.isna().sum())
print(medicamentos.NOMBRE_MEDICAMENTO.isnull().sum())

978
0
0


In [123]:
### CODIGO_DIAGNOSTICO EPS

print(medicamentos.CODIGO_DIAGNOSTICO_EPS_OP.drop_duplicates().count())
print(medicamentos.CODIGO_DIAGNOSTICO_EPS_OP.isna().sum())
print(medicamentos.CODIGO_DIAGNOSTICO_EPS_OP.isnull().sum())
medicamentos.CODIGO_DIAGNOSTICO_EPS_OP.head()


1187
0
0


0    L309
1    J304
2    D848
3    N760
4    Z929
Name: CODIGO_DIAGNOSTICO_EPS_OP, dtype: object

In [124]:
## Diagnostico Hipertension parece ser Rinitis Alergica y Urticaria

print(medicamentos.DIAGNOSTICO_EPS_DESC.drop_duplicates().count())
print(medicamentos.DIAGNOSTICO_EPS_DESC.isna().sum())
print(medicamentos.DIAGNOSTICO_EPS_DESC.isnull().sum())
medicamentos.DIAGNOSTICO_EPS_DESC.value_counts().head(10)

1370
0
0


ASMA PREDOMINANTEMENTE ALÉRGICA                            21149
ASMA, NO ESPECIFICADA                                      11032
HIPERTENSIÓN ESENCIAL (PRIMARIA)                           10415
RINITIS ALÉRGICA, NO ESPECIFICADA                           5300
HISTORIA PERSONAL DE TRATAMIENTO MÉDICO NO ESPECIFICADO     4616
URTICARIA, NO ESPECIFICADA                                  3015
OTRAS URTICARIAS                                            2421
URTICARIA IDIOPÁTICA                                        1600
ASMA PREDOMINANTEMENTE AL?RGICA                             1469
HIPOTIROIDISMO, NO ESPECIFICADO                             1439
Name: DIAGNOSTICO_EPS_DESC, dtype: int64

In [125]:
print(medicamentos.NUMERO_CANTIDAD_PRESTACIONES.isna().sum())
print(medicamentos.NUMERO_CANTIDAD_PRESTACIONES.isnull().sum())

medicamentos[medicamentos.NUMERO_CANTIDAD_PRESTACIONES.isna()].FECHA_EMISION.agg(["min","max"])
### Numerp de cantidad de prestaciones debe ser imputado para esta columna, poner comentario de imputación 

1253
1253


min   2020-01-20
max   2020-01-27
Name: FECHA_EMISION, dtype: datetime64[ns]

In [126]:
medicamentos[(medicamentos.FECHA_EMISION>=pd.to_datetime("2020-01-20"))&(medicamentos.FECHA_EMISION<=pd.to_datetime("2020-01-27"))].REGIONAL_EPS_DESC.value_counts()

Medellin        2233
Bogota           110
Manizales        107
Cali              95
Barranquilla      94
Pereira           40
Armenia           28
Name: REGIONAL_EPS_DESC, dtype: int64

In [127]:
### Las siguientes columnas de procesamiento y limpieza se deben añadir: Imputacion del campo NUMERO_CANTIDAD_PRESTACIONES
### Una Dummy que indique si hay imputacion en el anterior
### una columna de agrupamiento de los medicamentos que nos interesan

## Dejar listo para los script que crean las bases a discutir



In [128]:
medicamentos.columns
medicamentos=medicamentos.groupby(["ID_PACIENTE",'FECHA_EMISION','CODIGO_PRESTACION_OP']).agg("max").reset_index()

In [129]:
def dum_sign(dummy_col, threshold=0.1):

    # removes the bind
    dummy_col = dummy_col.copy()

    # what is the ratio of a dummy in whole column
    count = pd.value_counts(dummy_col) / len(dummy_col)

    # cond whether the ratios is higher than the threshold
    mask = dummy_col.isin(count[count > threshold].index)

    # replace the ones which ratio is lower than the threshold by a special name
    dummy_col[~mask] = "others"

    return pd.get_dummies(dummy_col, prefix=dummy_col.name)



#medicamentos2=pd.get_dummies(medicamentos)
#medicamentos2.FECHA_EMISION=medicamentos2.FECHA_EMISION.apply(lambda x: x.timestamp())

#medicamentos2.sum(axis=1)

#medicamentos_imputed=IterativeImputer(random_state=141854)
#medicamentos_imputed.fit_transform(medicamentos2)

In [130]:
medicamentos.columns

dum_reg=pd.get_dummies(medicamentos.REGIONAL_EPS_DESC)
dum_medicamento=dum_sign(medicamentos.NOMBRE_MEDICAMENTO,0.01)
dum_diag=dum_sign(medicamentos.DIAGNOSTICO_EPS_DESC,0.01)

aver=pd.concat([dum_reg,dum_medicamento,dum_diag,
                medicamentos.FECHA_EMISION.apply(lambda x: x.timestamp()),
                medicamentos.NUMERO_CANTIDAD_PRESTACIONES],axis=1)

medicamentos_imputed=IterativeImputer(random_state=141854)
sal=medicamentos_imputed.fit_transform(aver)

In [131]:
medicamentos["NumeroCantidadPrestacionesImputado"]=sal[:,-1]

In [132]:
medicamentos["NumeroCantidadPrestacionesImputadoInd"]=pd.isna(medicamentos.NUMERO_CANTIDAD_PRESTACIONES).astype(int)

In [133]:
medicamentos[medicamentos.NumeroCantidadPrestacionesImputadoInd==1]

,ID_PACIENTE,FECHA_EMISION,CODIGO_PRESTACION_OP,REGIONAL_EPS_DESC,NOMBRE_MEDICAMENTO,CODIGO_DIAGNOSTICO_EPS_OP,DIAGNOSTICO_EPS_DESC,NUMERO_CANTIDAD_PRESTACIONES,NumeroCantidadPrestacionesImputado,NumeroCantidadPrestacionesImputadoInd


In [134]:
## El objetivo de estos script es crear los casos de observación de adherencia. 
## La metodologia consiste en ver si existe otra entrega de medicación cercana al dia final 
## para las entregas de medicacion (un delta de 5 dias es posible)
## Existen casos y lecturas que complican la situación se puede añadir una lectura de "seguridad"
## que es 1 si el medicamento es el mismo o 0 si solo existe otro medicamento.

In [135]:
diagnosticos=pd.read_excel("Diagnostico.xlsx")
diagnosticos.drop(columns="Unnamed: 0",inplace=True)

In [136]:
medicamentos=medicamentos.merge(diagnosticos,how="left")

In [137]:
tipo_med=pd.read_excel("Unico_Medicamento.xlsx")

In [138]:
tipo_med.columns

Index(['NOMBRE_MEDICAMENTO', 'Categoria', 'Tratamiento'], dtype='object')

In [139]:
medicamentos=medicamentos.merge(tipo_med,how="left")

In [140]:
print(medicamentos.shape)
print(medicamentos.columns)


(108086, 13)
Index(['ID_PACIENTE', 'FECHA_EMISION', 'CODIGO_PRESTACION_OP',
       'REGIONAL_EPS_DESC', 'NOMBRE_MEDICAMENTO', 'CODIGO_DIAGNOSTICO_EPS_OP',
       'DIAGNOSTICO_EPS_DESC', 'NUMERO_CANTIDAD_PRESTACIONES',
       'NumeroCantidadPrestacionesImputado',
       'NumeroCantidadPrestacionesImputadoInd', 'DiagnosticoAgrupadoRel',
       'Categoria', 'Tratamiento'],
      dtype='object')


In [141]:
medicamentos.to_excel("medicamentos_limpio_19jun.xlsx")